In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

df_train = pd.read_csv('train.csv', index_col='id')
df_test = pd.read_csv('test.csv', index_col='id')

sample_submission = pd.read_csv('sample_submission.csv', index_col='id')

In [4]:
binvar = ['bin_' + str(i) for i in range(1,5)]
ordvar = ['ord_' + str(i) for i in range(6)]
nomvar = ['nom_' + str(i) for i in range(10)]
dmvar  = ['day', 'month']

df_work = df_train.copy()
y_train = df_work['target'].copy()
df_work = pd.concat([df_work.drop('target', axis = 1), df_test])

df_work.drop('bin_0', inplace=True, axis=1)

df_work['bin_3'] = df_work['bin_3'].map({'F':0, 'T':1})
df_work['bin_4'] = df_work['bin_4'].map({'N':0, 'Y':1})

df_work['ord_0'] = df_work['ord_0'] - 1

In [7]:
df_work

,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,...,c389000ab,2f4cb3d51,1,Grandmaster,Cold,h,D,kr,2,2
1,1,0,1,1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,...,4cd920251,f83c56c21,0,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,1,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,...,de9c9f684,ae6800dd0,0,Expert,Lava Hot,h,R,Jc,7,2
3,1,0,0,1,Red,Trapezoid,Snake,Canada,Oboe,f12246592,...,4ade6ab69,8270f0d71,0,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,0,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,...,cb43ab175,b164b72a7,0,Grandmaster,Freezing,a,R,qP,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,0,Green,Square,Lion,Canada,Theremin,6c90f81cd,...,9e4b23160,acc31291f,0,Novice,Lava Hot,j,A,Gb,1,3
499996,0,0,0,1,Green,Trapezoid,Lion,China,Piano,2d61990e2,...,cfbd87ed0,eae3446d0,0,Contributor,Lava Hot,f,S,Ed,2,2
499997,1,1,1,1,Green,Trapezoid,Lion,Canada,Oboe,488406659,...,1108bcd6c,33dd3cf4b,0,Novice,Boiling Hot,g,V,TR,3,1


In [8]:
ord1dict = {'Novice':0, 'Contributor':1, 'Expert':2, 'Master':3, 'Grandmaster':4}
df_work['ord_1'] = df_work['ord_1'].map(ord1dict)

ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}
df_work['ord_2'] = df_work['ord_2'].map(ord2dict)

oe = OrdinalEncoder(categories='auto')
df_work[ordvar[3:]] = oe.fit_transform(df_work[ordvar[3:]])
for var, cl in zip(ordvar[3:], oe.categories_):
    print(var)
    print(cl)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AP' 'Ai' 'Aj' 'BA' 'BE' 'Bb' 'Bd' 'Bn' 'CL' 'CM' 'CU' 'CZ' 'Cl' 'DH'
 'DN' 'Dc' 'Dx' 'Ed' 'Eg' 'Er' 'FI' 'Fd' 'Fo' 'GD' 'GJ' 'Gb' 'Gx' 'Hj'
 'IK' 'Id' 'JX' 'Jc' 'Jf' 'Jt' 'KR' 'KZ' 'Kf' 'Kq' 'LE' 'MC' 'MO' 'MV'
 'Mf' 'Ml' 'Mx' 'NV' 'Nf' 'Nk' 'OR' 'Ob' 'Os' 'PA' 'PQ' 'PZ' 'Ps' 'QM'
 'Qb' 'Qh' 'Qo' 'RG' 'RL' 'RP' 'Rm' 'Ry' 'SB' 'Sc' 'TR' 'TZ' 'To' 'UO'
 'Uk' 'Uu' 'Vf' 'Vx' 'WE' 'Wc' 'Wv' 'XI' 'Xh' 'Xi' 'YC' 'Yb' 'Ye' 'ZR'
 'ZS' 'Zc' 'Zq' 'aF' 'aM' 'aO' 'aP' 'ac' 'av' 'bF' 'bJ' 'be' 'cA' 'cG'
 'cW' 'ck' 'cp' 'dB' 'dE' 'dN' 'dO' 'dP' 'dQ' 'dZ' 'dh' 'eG' 'eQ' 'eb'
 'eg' 'ek' 'ex' 'fO' 'fh' 'gJ' 'gM' 'hL' 'hT' 'hh' 'hp' 'iT' 'ih' 'jS'
 'jV' 'je' 'jp' 'kC' 'kE' 'kK' 'kL' 'kU' 'kW' 'ke' 'kr' 'kw' 'lF' 'lL'
 'll' 'lx' 'mb' 'mc' 'mm' 'nX' 'nh' 'oC' 'oG' 'oH' 'oK' 'od' 'on' 'pa'
 'ps' 'qA' 'qJ' 'qK' 'qP' 'qX' '

In [9]:
df_work[ordvar] = StandardScaler().fit_transform(df_work[ordvar])
df_work

,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,...,c389000ab,2f4cb3d51,0.731992,1.416421,-0.660160,0.100961,-1.338303,0.757446,2,2
1,1,0,1,1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,...,4cd920251,f83c56c21,-0.672627,1.416421,0.324194,-1.726851,-1.748107,-0.018723,7,8
2,0,0,0,1,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,...,de9c9f684,ae6800dd0,-0.672627,0.224377,1.308547,0.100961,0.574119,-1.137850,7,2
3,1,0,0,1,Red,Trapezoid,Snake,Canada,Oboe,f12246592,...,4ade6ab69,8270f0d71,-0.672627,1.416421,0.816371,0.362077,-1.338303,0.721345,2,1
4,0,0,0,0,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,...,cb43ab175,b164b72a7,-0.672627,1.416421,-1.152336,-1.726851,0.574119,1.154555,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,0,Green,Square,Lion,Canada,Theremin,6c90f81cd,...,9e4b23160,acc31291f,-0.672627,-0.967667,1.308547,0.623193,-1.748107,-1.246153,1,3
499996,0,0,0,1,Green,Trapezoid,Lion,China,Piano,2d61990e2,...,cfbd87ed0,eae3446d0,-0.672627,-0.371645,1.308547,-0.421271,0.710720,-1.390556,2,2
499997,1,1,1,1,Green,Trapezoid,Lion,Canada,Oboe,488406659,...,1108bcd6c,33dd3cf4b,-0.672627,-0.967667,0.816371,-0.160155,1.120525,-0.506085,3,1


In [10]:
df_work[nomvar[5:]].nunique()


nom_5      222
nom_6      522
nom_7     1220
nom_8     2219
nom_9    12068
dtype: int64

In [11]:
df_work['nom_5'] = df_work['nom_5'].str[4:]
df_work['nom_6'] = df_work['nom_6'].str[3:]
df_work['nom_7'] = df_work['nom_7'].str[3:]
df_work['nom_8'] = df_work['nom_8'].str[3:]
df_work['nom_9'] = df_work['nom_9'].str[3:]
df_work[nomvar[5:]].nunique()

nom_5      222
nom_6      522
nom_7     1220
nom_8     2218
nom_9    12064
dtype: int64

In [12]:
enc = OneHotEncoder(categories = 'auto', dtype = 'float64', drop = 'first')
nom_matrix = enc.fit_transform(df_work[nomvar])
df_work.drop(nomvar, inplace=True, axis=1)

enc = OneHotEncoder(categories = 'auto', dtype = 'float64', drop = 'first')
dm_matrix = enc.fit_transform(df_work[dmvar])
df_work.drop(dmvar, inplace=True, axis=1)

In [13]:
df_work.columns

Index(['bin_1', 'bin_2', 'bin_3', 'bin_4', 'ord_0', 'ord_1', 'ord_2', 'ord_3',
       'ord_4', 'ord_5'],
      dtype='object')

In [15]:
df_work_sprs =scipy.sparse.hstack([nom_matrix, scipy.sparse.coo_matrix(df_work).astype('float64'), dm_matrix]).tocsr()
display(df_work_sprs)

<500000x16288 sparse matrix of type '<class 'numpy.float64'>'
	with 9235121 stored elements in Compressed Sparse Row format>

In [16]:
X_train = df_work_sprs[:y_train.shape[0]]
X_test = df_work_sprs[y_train.shape[0]:]

In [24]:
%%time

for i in np.arange(0.04, 0.21, 0.01):
    C = i
    
    clf = LogisticRegression(C=C, solver='lbfgs', max_iter=1000, verbose=0, n_jobs=-1)
    
    
    score = cross_validate(clf, X_train, y_train, cv=3, scoring="roc_auc")
    mean = score['test_score'].mean()
    print(score['test_score'])
    print('C =', C, f'{mean:.8f}')

[0.79806598 0.80287577 0.7998293 ]
C = 0.04 0.80025701
[0.79894227 0.8036727  0.80076884]
C = 0.05 0.80112794
[0.79951778 0.80417771 0.80138746]
C = 0.060000000000000005 0.80169432
[0.79990843 0.80450391 0.80180143]
C = 0.07 0.80207126
[0.80017086 0.80470981 0.80207605]
C = 0.08000000000000002 0.80231891
[0.8003405  0.80483518 0.80225833]
C = 0.09000000000000001 0.80247800
[0.80044695 0.80490055 0.80237288]
C = 0.1 0.80257346
[0.80050791 0.80492035 0.80243548]
C = 0.11000000000000001 0.80262125
[0.80053159 0.80490766 0.80245892]
C = 0.12000000000000002 0.80263273
[0.80052556 0.80487081 0.80244915]
C = 0.13000000000000003 0.80261517
[0.80049777 0.80480923 0.80242238]
C = 0.14 0.80257646
[0.80045623 0.80473378 0.80237386]
C = 0.15000000000000002 0.80252129
[0.8003949  0.80464929 0.80231042]
C = 0.16000000000000003 0.80245154
[0.80032355 0.80455004 0.80223104]
C = 0.17000000000000004 0.80236821
[0.80024413 0.80444691 0.80214918]
C = 0.18000000000000002 0.80228007
[0.8001553  0.8043345  0.

In [37]:
%%time

clf = LogisticRegression(C=0.005, solver='lbfgs', max_iter=1000, verbose=0, n_jobs=-1)
clf.fit(X_train, y_train)

Wall time: 6.83 s


LogisticRegression(C=0.005, max_iter=1000, n_jobs=-1)

In [38]:
y_preds = clf.predict_proba(X_test)[:,1]

In [39]:
sample_submission['target'] = y_preds
sample_submission.to_csv('submission_4.csv')